In [26]:
import pandas as pd
import re

def parse_intermediate_range(i_val):
    """
    Helper function to parse the 'I' or 'ATU' column.
    Handles '18-19', '18', and '-'
    """
    i_val = str(i_val).strip()
    range_match = re.match(r"(\d+)-(\d+)", i_val)
    if range_match:
        return int(range_match.group(1)), int(range_match.group(2))
    
    single_num_match = re.fullmatch(r"(\d+)", i_val)
    if single_num_match:
        num = int(single_num_match.group(1))
        return num, num # Min and Max are the same
    
    return None, None

def parse_breakpoint_value(val):
    """
    Helper function to extract the first number from S or R columns.
    Handles '>= 18' or '< 17' or just '18'
    """
    val = str(val).strip()
    num_match = re.search(r"(\d+)", val)
    if num_match:
        return int(num_match.group(1))
    return None

# --- THIS FUNCTION IS NEW AND CORRECTED ---
def find_header_rows(df):
    """
    Scans the raw DataFrame to find the row index of the *second* header row
    (the one with "S ≥" and "R <").
    """
    # Check the first 15 rows
    for i, row in df.head(15).iterrows():
        found_S = False
        found_R = False
        
        # Check every cell in the row for the exact strings
        for cell in row:
            cell_str = str(cell).strip()
            if cell_str == "S ≥":
                found_S = True
            if cell_str == "R <":
                found_R = True
        
        # If both keywords are in the same row, this is our second header row
        if found_S and found_R:
            return i - 1  # Return the index of the *first* header row
            
    return None # Did not find the header row
# --- END NEW FUNCTION ---

def extract_all_breakpoints():
    """
    Reads the Excel file, dynamically finds the header on each sheet,
    and extracts the S/I/R data.
    """
    
    excel_file_path = "v_15.0_Breakpoint_Tables.xlsx"

    BLACKLIST_SHEETS = [
        "Content", "Changes", "Notes", "Guidance", "Dosages", 
        "Technical uncertainty", "PK PD breakpoints", "PK PD cutoffs",
        "Topical agents", "M.tuberculosis", "L.pneumophila"
    ]

    all_breakpoint_data = []

    try:
        try:
            xls = pd.ExcelFile(excel_file_path)
        except FileNotFoundError:
            print(f"ERROR: File not found at '{excel_file_path}'")
            return []

        all_sheet_names = xls.sheet_names
        print(f"Found {len(all_sheet_names)} sheets. Processing...")

        for sheet_name in all_sheet_names:
            
            if sheet_name.strip() in BLACKLIST_SHEETS:
                print(f"-> Skipping sheet (in blacklist): {sheet_name}")
                continue

            microbe_name = sheet_name.strip()
            print(f"Processing data for: {microbe_name}")

            try:
                # 1. Read the sheet with NO header
                df_raw = pd.read_excel(xls, sheet_name=sheet_name, header=None)

                # 2. Find the index of the header row (using the NEW function)
                header_index = find_header_rows(df_raw)
                
                if header_index is None:
                    print(f"  -> Could not find 'S ≥' and 'R <' header row in {microbe_name}. Skipping.")
                    continue
                
                # 3. Re-create the DataFrame with the correct header
                header_1 = df_raw.iloc[header_index].ffill()
                header_2 = df_raw.iloc[header_index + 1]
                df_columns = pd.MultiIndex.from_arrays([header_1, header_2])
                
                df_data = df_raw.iloc[header_index + 2:]
                df_data.columns = df_columns
                df_data = df_data.reset_index(drop=True)

                # --- 4. Dynamic Column Finding Logic ---
                s_col, r_col, i_col = None, None, None
                antibiotic_col = df_data.columns[0] # Assume first col is antibiotic

                for col in df_data.columns:
                    parent_header = str(col[0])
                    sub_header = str(col[1]).strip()
                    
                    if "Zone diameter breakpoints (mm)" in parent_header or "Disk diffusion" in parent_header:
                        
                        # --- THIS IS THE FIX ---
                        if sub_header == "S ≥": s_col = col
                        if sub_header == "R <": r_col = col
                        if sub_header == "ATU": i_col = col
                        # ---
                        
                        # Fallback for other formats
                        if "S (>= mm)" == sub_header: s_col = col
                        if "R (<= mm)" == sub_header: r_col = col
                        if "I (mm)" == sub_header: i_col = col
                        if "ATU S >=" == sub_header: s_col = col
                        if "ATU B" == sub_header: i_col = col

                if not s_col or not r_col:
                    print(f"  -> S/R breakpoint columns not found. Skipping.")
                    continue

                # --- 5. Extract Data ---
                columns_to_extract = [antibiotic_col, s_col, r_col]
                column_names = ['Antibiotic', 'S_col', 'R_col']
                
                if i_col:
                    columns_to_extract.append(i_col)
                    column_names.append('I_col')

                data_subset = df_data[columns_to_extract].copy()
                data_subset.columns = column_names
                data_subset = data_subset.dropna(subset=['Antibiotic'])

                for index, row in data_subset.iterrows():
                    medicine_name = str(row['Antibiotic']).strip()
                    # Skip rows that are just headers or notes
                    if medicine_name.lower() in ["antibiotic", "penicillins", "cephalosporins", "carbapenems", "monobactams", "fluoroquinolones", "aminoglycosides", "glycopeptides and lipoglycopeptides", "macrolides, lincosamides and streptogramins", "tetracyclines", "oxazolidinones", "miscellaneous agents"]:
                        continue 

                    s_min = parse_breakpoint_value(row['S_col'])
                    
                    r_raw_val = parse_breakpoint_value(row['R_col'])
                    r_max = None
                    if r_raw_val is not None:
                        r_col_name = str(r_col[1]).strip() # Check the sub-header name
                        if r_col_name == "R <" or r_col_name == "ATU R <":
                            r_max = r_raw_val - 1 # Logic: "R < 18" means R is <= 17
                        else:
                            r_max = r_raw_val # Logic: "R (<= mm)" or just "17"
                    
                    i_min, i_max = (None, None)
                    if 'I_col' in row:
                        i_min, i_max = parse_intermediate_range(row['I_col'])

                    all_breakpoint_data.append({
                        "microbe_name": microbe_name,
                        "medicine_name": medicine_name,
                        "susceptible_min_mm": s_min,
                        "resistant_max_mm": r_max,
                        "intermediate_min_mm": i_min,
                        "intermediate_max_mm": i_max
                    })

            except Exception as e:
                print(f"  -> ERROR processing sheet {sheet_name}: {e}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

    print("\n--- Data Extraction Complete ---")
    return all_breakpoint_data

# --- Run the function ---
structured_data = extract_all_breakpoints()

if structured_data:
    print(f"\nTotal rules extracted: {len(structured_data)}")
    
    # Save to a new, clean CSV
    final_df = pd.DataFrame(structured_data)
    final_df.to_csv("cleaned_breakpoints_final.csv", index=False)
    print("\nSuccessfully saved all data to 'cleaned_breakpoints_final.csv'")
    
    print("\n--- First 5 Entries ---")
    print(final_df.head())

Found 41 sheets. Processing...
-> Skipping sheet (in blacklist): Content
-> Skipping sheet (in blacklist): Changes
-> Skipping sheet (in blacklist): Notes
-> Skipping sheet (in blacklist): Guidance
-> Skipping sheet (in blacklist): Dosages
-> Skipping sheet (in blacklist): Technical uncertainty
Processing data for: Enterobacterales
Processing data for: Pseudomonas
Processing data for: S.maltophilia
Processing data for: Acinetobacter
Processing data for: Staphylococcus
Processing data for: Enterococcus
Processing data for: Streptococcus A,B,C,G
Processing data for: S.pneumoniae
Processing data for: Viridans group streptococci
Processing data for: H.influenzae
Processing data for: M.catarrhalis
Processing data for: N.gonorrhoeae
  -> Could not find 'S ≥' and 'R <' header row in N.gonorrhoeae. Skipping.
Processing data for: N.meningitidis
  -> Could not find 'S ≥' and 'R <' header row in N.meningitidis. Skipping.
Processing data for: Anaerobic bacteria
Processing data for: H.pylori
  -> C

In [27]:
structured_data

[{'microbe_name': 'Enterobacterales',
  'medicine_name': 'Benzylpenicillin',
  'susceptible_min_mm': None,
  'resistant_max_mm': None,
  'intermediate_min_mm': None,
  'intermediate_max_mm': None},
 {'microbe_name': 'Enterobacterales',
  'medicine_name': 'Ampicillin iv1',
  'susceptible_min_mm': 14,
  'resistant_max_mm': 13,
  'intermediate_min_mm': None,
  'intermediate_max_mm': None},
 {'microbe_name': 'Enterobacterales',
  'medicine_name': 'Ampicillin oral (uncomplicated UTI only)1',
  'susceptible_min_mm': 14,
  'resistant_max_mm': 13,
  'intermediate_min_mm': None,
  'intermediate_max_mm': None},
 {'microbe_name': 'Enterobacterales',
  'medicine_name': 'Ampicillin-sulbactam iv1',
  'susceptible_min_mm': 14,
  'resistant_max_mm': 13,
  'intermediate_min_mm': None,
  'intermediate_max_mm': None},
 {'microbe_name': 'Enterobacterales',
  'medicine_name': 'Ampicillin-sulbactam oral (uncomplicated UTI only)1',
  'susceptible_min_mm': 14,
  'resistant_max_mm': 13,
  'intermediate_min_mm'

In [25]:
extract_all_breakpoints()

Found 41 sheets. Processing...
-> Skipping sheet (in blacklist): Content
-> Skipping sheet (in blacklist): Changes
-> Skipping sheet (in blacklist): Notes
-> Skipping sheet (in blacklist): Guidance
-> Skipping sheet (in blacklist): Dosages
-> Skipping sheet (in blacklist): Technical uncertainty
Processing data for: Enterobacterales
  -> S/R breakpoint columns not found. Skipping.
Processing data for: Pseudomonas
  -> S/R breakpoint columns not found. Skipping.
Processing data for: S.maltophilia
  -> S/R breakpoint columns not found. Skipping.
Processing data for: Acinetobacter
  -> S/R breakpoint columns not found. Skipping.
Processing data for: Staphylococcus
  -> S/R breakpoint columns not found. Skipping.
Processing data for: Enterococcus
  -> S/R breakpoint columns not found. Skipping.
Processing data for: Streptococcus A,B,C,G
  -> S/R breakpoint columns not found. Skipping.
Processing data for: S.pneumoniae
  -> S/R breakpoint columns not found. Skipping.
Processing data for: Vir

[]